In [113]:
import datetime
import pandas as pd
import random
import numpy as np
import seaborn as sns

# Set display format to avoid scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [116]:
rng = np.random.default_rng(123)

start_year = 1950
end_year = 2050
portfolios = ["First", "Second", "Third", "Fourth", "Fifth"]
portfolios_list = []
dates_list = []


for portfolio in portfolios:
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            date = datetime.date(year, month, 1)
            dates_list.append(date)
            portfolios_list.append(portfolio)

random_returns = rng.uniform(low=(-0.1 / 12), high=(0.3 / 12), size=len(dates_list))


fake_returns = pd.DataFrame(
    {"Portfolio": portfolios_list, "date": dates_list, "ret": random_returns}
)


In [3]:
fake_returns

,Portfolio,date,ret
0,First,1950-01-01,0.014412
1,First,1950-02-01,-0.006539
2,First,1950-03-01,-0.000988
3,First,1950-04-01,-0.002188
4,First,1950-05-01,-0.002470
...,...,...,...
4255,Fifth,2020-08-01,0.002473
4256,Fifth,2020-09-01,0.004191
4257,Fifth,2020-10-01,-0.007476
4258,Fifth,2020-11-01,0.005397


In [117]:
# Some pseudo code

# Input variables (these require input from the user, MANDATORY)

submitted_income = 85000
income_growth = 0.035

monthly_income = submitted_income/12
monthly_growth = 1+ (income_growth/12)

start_savings = 25
retirement_start = 65
death_year = 85

month_start_savings = start_savings*12
month_retirement_start = retirement_start*12
death_month = death_year*12



# I think I just want these to be static
save_rate = 0.10
consumption_rate = 0.04

In [118]:
total_monthly_incomes = []

for t in fake_returns.index[month_start_savings:month_retirement_start]:
    current_month = t-month_start_savings
    income_for_month = monthly_income*(monthly_growth**current_month)
    total_monthly_incomes.append(income_for_month)


In [119]:
# Apply mapping to DataFrame's index to create a month column that resets for each portfolio
fake_returns['month'] = fake_returns.groupby('Portfolio').cumcount()

# Map the total_monthly_incomes to each row based on the month
income_series = pd.Series(total_monthly_incomes, index=range(month_start_savings, month_retirement_start))
fake_returns['income'] = fake_returns['month'].map(income_series)

# Optional: Remove the month column if it's no longer needed
#fake_returns.drop(columns='month', inplace=True)


In [121]:
fake_returns['savings'] = 0.0

for portfolio in fake_returns['Portfolio'].unique():
    # Filter the DataFrame by portfolio
    portfolio_data = fake_returns[fake_returns['Portfolio'] == portfolio]
    
    # Initialize previous savings
    previous_savings = 0
    
    # Iterate through each row in the portfolio data
    for index, row in portfolio_data.iterrows():
        if pd.isna(row['income']):
            current_savings = previous_savings * (1 + row['ret'])
            fake_returns.at[index, 'savings'] = current_savings
        else:
            current_savings = save_rate * row['income'] + previous_savings * (1 + row['ret'])
            fake_returns.at[index, 'savings'] = current_savings
        # Update previous savings
        previous_savings = current_savings

In [125]:
portfolio_data

,Portfolio,date,ret,month,income,savings
4848,Fifth,1950-01-01,0.011,0,NaN,0.000
4849,Fifth,1950-02-01,0.017,1,NaN,0.000
4850,Fifth,1950-03-01,-0.008,2,NaN,0.000
4851,Fifth,1950-04-01,0.025,3,NaN,0.000
4852,Fifth,1950-05-01,0.020,4,NaN,0.000
...,...,...,...,...,...,...
6055,Fifth,2050-08-01,0.021,1207,NaN,0.000
6056,Fifth,2050-09-01,0.006,1208,NaN,0.000
6057,Fifth,2050-10-01,0.014,1209,NaN,0.000
6058,Fifth,2050-11-01,-0.003,1210,NaN,0.000


In [105]:
print(5+current_savings)

nan


In [124]:
fake_returns.iloc[month_start_savings:month_retirement_start]

,Portfolio,date,ret,month,income,savings
300,First,1975-01-01,0.001,300,7083.333,708.333
301,First,1975-02-01,0.010,301,7103.993,1425.902
302,First,1975-03-01,0.010,302,7124.713,2152.069
303,First,1975-04-01,0.006,303,7145.493,2879.655
304,First,1975-05-01,0.003,304,7166.334,3606.202
...,...,...,...,...,...,...
775,First,2014-08-01,0.003,775,28251.450,6891313.762
776,First,2014-09-01,0.021,776,28333.850,7041930.140
777,First,2014-10-01,0.025,777,28416.491,7220728.175
778,First,2014-11-01,0.020,778,28499.372,7365840.762


In [123]:
print(fake_returns.to_string())

     Portfolio        date    ret  month    income       savings
0        First  1950-01-01  0.014      0       NaN         0.000
1        First  1950-02-01 -0.007      1       NaN         0.000
2        First  1950-03-01 -0.001      2       NaN         0.000
3        First  1950-04-01 -0.002      3       NaN         0.000
4        First  1950-05-01 -0.002      4       NaN         0.000
5        First  1950-06-01  0.019      5       NaN         0.000
6        First  1950-07-01  0.022      6       NaN         0.000
7        First  1950-08-01  0.001      7       NaN         0.000
8        First  1950-09-01  0.019      8       NaN         0.000
9        First  1950-10-01  0.021      9       NaN         0.000
10       First  1950-11-01  0.009     10       NaN         0.000
11       First  1950-12-01 -0.000     11       NaN         0.000
12       First  1951-01-01  0.019     12       NaN         0.000
13       First  1951-02-01 -0.001     13       NaN         0.000
14       First  1951-03-0

In [47]:
test_df['savings'].count()

0